# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2025/11/11/

In [7]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [8]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'company page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'blog', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'portfolio', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'portfolio', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook page',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [9]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [ ]:
select_relevant_links("https://edwarddonner.com")

In [10]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 14 relevant links


{'links': [{'type': 'enterprise page',
   'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'join page', 'url': 'https://huggingface.co/join'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'endpoints product page',
   'url': 'https://endpoints.huggingface.co'},
  {'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'learn page', 'url': 'https://huggingface.co/learn'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'community forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'Status page', 'url': 'https://status.huggingface.co/'},
  {'type': 'Discord', 'url': 'https://huggingface.co/join/discord'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [11]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [12]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
Lightricks/LTX-2
Updated
4 days ago
•
630k
•
801
tencent/HY-MT1.5-1.8B
Updated
11 days ago
•
9.77k
•
719
fal/Qwen-Image-Edit-2511-Multiple-Angles-LoRA
Updated
5 days ago
•
16k
•
452
nvidia/nemotron-speech-streaming-en-0.6b
Updated
6 days ago
•
2.26k
•
301
LiquidAI/LFM2.5-1.2B-Instruct
Updated
3 days ago
•
11.6k
•
272
Browse 2M+ models
Spaces
Running
Featured
4.14k
Wan2.2 Animate
👁
4.14k
Wan2.2 Animate
Running
on
Zero
463
Qwen Image Multiple Angles 3D Camera
🎥
463
Adjust camera angles in images using 3D controls or sliders
Running
on
Zero
1.31k
Z Image Turbo
🖼
1.31k
Generate stunning images from text descriptions in seconds
Runnin

In [13]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [14]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [15]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 8 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nLightricks/LTX-2\nUpdated\n4 days ago\n•\n630k\n•\n801\ntencent/HY-MT1.5-1.8B\nUpdated\n11 days ago\n•\n9.77k\n•\n719\nfal/Qwen-Image-Edit-2511-Multiple-Angles-LoRA\nUpdated\n5 days ago\n•\n16k\n•\n452\nnvidia/nemotron-speech-streaming-en-0.6b\nUpdated\n6 days ago\n•\n2.26k\n•\n301\nLiquidAI/LFM2.5-1.2B-Instruct\nUpdated\n3 days ago\n•\n11.6k\n•\n272\nBrowse 2M+ models\nSpaces\nRunning\nFea

In [16]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the vibrant AI community building the future of machine learning. It is the premier platform where researchers, developers, and organizations collaborate to create, share, and deploy AI models, datasets, and applications. With over 2 million models and 500,000+ datasets available, Hugging Face empowers the global machine learning community to accelerate AI innovation across multiple modalities including text, image, video, audio, and 3D.

---

## What We Offer

### Collaboration Platform
- **Host and collaborate** on unlimited public models, datasets, and applications.
- Engage with an active community to discover cutting-edge ML assets.
- Share your work to build a visible ML profile and portfolio.

### Models & Applications
- Explore groundbreaking models like Lightricks/LTX-2, Tencent’s HY-MT series, Nvidia's speech streaming, and many more.
- Access over **2 million models** and a continuously growing database of **1 million+ applications**.
- Run and deploy applications instantly via **Spaces**, the platform’s playground for AI demos.

### Datasets
- Unlock access to over **500,000 datasets** curated and updated regularly by communities and partners globally.
- Support for diverse data types ideal for tasks in autonomous vehicles, finance, offensive security, multilingual translation, and more.

### Open Source Stack
- Utilize the Hugging Face open-source tools and architecture designed to speed development, experimentation, and deployment.
- Supports seamless integration of AI pipelines with enterprise-grade reliability.

---

## For Enterprises

Hugging Face offers tailored AI platform solutions for teams and enterprises, including:
- Advanced security features such as **Single Sign-On (SSO)** and **comprehensive audit logs**.
- Flexible region selection for repository data to meet compliance needs.
- Dedicated support and scalable team plans starting at $20/user/month.
- Custom enterprise contracts for organizations needing bespoke solutions.

---

## Company Culture

- **Community-Driven:** Hugging Face thrives on open collaboration and shared innovation across a global network of AI practitioners.
- **Inclusive and Empowering:** The platform encourages people at all skill levels to contribute, learn, and grow together.
- **Transparency and Openness:** Leadership embraces open source, open science, and openness in communication.
- **Innovation Focus:** Constantly pushing the boundaries of what’s possible with AI across modalities.

---

## Customers & Community

Hugging Face serves a diverse user base:
- Individual researchers and developers building and sharing cutting-edge ML models.
- Startups and enterprises accelerating product innovation with AI.
- Academia and research institutions contributing to open scientific advancements.
- Industry leaders in fields such as autonomous vehicles, finance, creative applications, and more.

---

## Careers at Hugging Face

Joining Hugging Face means becoming part of a forward-thinking AI community prioritizing:
- **Creative problem solving** in AI/ML.
- **Open collaboration** with global researchers and engineers.
- **Continuous learning** and growth within a fast-paced innovative environment.

Prospective candidates can explore available roles focused on engineering, research, product, and community growth by visiting the Hugging Face careers page (typically found through the company website).

---

## Get Started

- Explore AI apps and models at [huggingface.co](https://huggingface.co)
- Sign up to host your models, datasets, and applications.
- Join the AI community accelerating machine learning innovation across the globe.

---

## Contact & Pricing

- Flexible plans for individuals, teams, and enterprises.
- Team subscriptions start at $20 per user/month.
- Enterprise options available with dedicated support and customized contracts.
- Contact sales for personalized consultations and demos.

---

**Hugging Face** – building the future of AI, together.  
Visit: [https://huggingface.co](https://huggingface.co)  
Join the community and accelerate your machine learning journey today!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 11 relevant links


# Hugging Face: The AI Community Building the Future

---

## About Hugging Face

Hugging Face is the premier collaboration platform designed for the global machine learning (ML) community. It serves as the **home of machine learning**, enabling ML engineers, scientists, and enthusiasts worldwide to **create, discover, and collaborate** on models, datasets, and ML-powered applications across all modalities including text, image, video, audio, and 3D.

At its core, Hugging Face empowers people to build, share, and experiment with open-source ML models and datasets, accelerating innovation and the development of an **open and ethical AI future**.

---

## What Hugging Face Offers

- **2M+ Models**  
  Access and contribute to a vast and ever-growing repository of state-of-the-art models, including trending models updated regularly by leading organizations such as NVIDIA, Tencent, and LiquidAI.

- **500K+ Datasets**  
  Discover and share diverse datasets for training and evaluation purposes, ranging from autonomous vehicle data to financial time series.

- **1M+ Applications (Spaces)**  
  Explore AI applications built by the community, including cutting-edge demos like photorealistic image generators, 3D image editors, and real-time animations.

- **Open Source Stack**  
  Utilize Hugging Face’s open-source tools and infrastructure to move faster and build better ML solutions together.

- **Community & Collaboration**  
  Participate in a vibrant and rapidly growing community where collaboration is fostered through unlimited hosting and sharing of models, datasets, and AI apps.

---

## Culture & Values

- **Open and Ethical AI**: Hugging Face is deeply committed to creating an ethical AI ecosystem where accessibility and transparency are paramount.

- **Community-Driven Innovation**: The platform thrives on community participation, allowing anyone to contribute, learn, and grow with others passionate about machine learning.

- **Support for Next-Generation ML Engineers and Scientists**: Hugging Face provides tools and an environment conducive to learning, building portfolios, and accelerating careers in AI.

- **Inclusivity & Accessibility**: By providing free access to thousands of resources and encouraging open collaboration, Hugging Face lowers the barrier to entry for AI practitioners worldwide.

---

## Customers & Users

Hugging Face serves a diverse set of users:

- **ML Engineers & Researchers**: Benefit from a rich repository of models and shared datasets for research and production.
- **Enterprise Clients**: Leverage enterprise-grade features and services designed to integrate Hugging Face’s tools into business applications.
- **Developers & AI Enthusiasts**: Experiment and build AI applications using ready-made or custom models.
- **Educational Institutions and Students**: Use Hugging Face as a learning platform to gain hands-on experience with cutting-edge AI.

---

## Careers & Opportunities

Join Hugging Face and become a part of a fast-growing, mission-driven company at the forefront of AI innovation.

- **Open Positions**: Roles across engineering, research, product, and community engagement.
- **Work Environment**: Collaborative, inclusive culture focused on impact and ethical AI development.
- **Growth & Development**: Access to a community of like-minded AI professionals and continuous learning opportunities.
- **Mission-Driven**: Contribute to building tools and platforms that democratize AI and promote open science.

---

## Connect & Get Started

- **Explore AI Apps and Models**: Visit Hugging Face’s platform to browse millions of models, datasets, and applications.  
- **Join the Community**: Sign up to share your work, collaborate with others, and build your ML profile.  
- **Enterprise Solutions**: Discover bespoke packages tailored to your business needs.

Explore the future of AI and machine learning with **Hugging Face – The AI community building the future.**

---

### Brand Highlights

- **Logo Colors:** Bright yellow (#FFD21E), orange (#FF9D00), and neutral gray (#6B7280)  
- **Brand Identity:** Friendly, approachable, and innovative—reflective of an open-source community that empowers cutting-edge AI development.

---

Discover more and join the AI revolution at **https://huggingface.co/**.

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>